In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# GCP上的端到端ML：MLOps第1阶段：数据管理：开始使用Dataflow

<table align="left">
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage1/get_started_dataflow.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
        <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage1/get_started_dataflow.ipynb">
        <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png\" alt="Colab logo"> 在Colab中运行
        </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/ml_ops/stage1/get_started_dataflow.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示了如何在Google Cloud上使用Vertex AI进行端到端MLOps生产环境。本教程涵盖了阶段1：数据管理：开始使用Dataflow。

### 目标

在本教程中，您将学习如何在`Vertex AI`中使用`Dataflow`进行培训。

本教程使用以下谷歌云ML服务:

- `Dataflow`
- `BigQuery数据集`

执行的步骤包括:

- 数据的离线预处理:
    - 串行 - 无Dataflow
    - 并行 - 使用Dataflow
- 上游数据的预处理:
    - 表格数据
    - 图像数据

### 推荐

在 Google Cloud 上进行端到端 MLOps 时，以下是预处理和训练自定义模型期间数据馈送的最佳实践：

#### 预处理

数据预处理可以是：

- 离线：数据在训练之前被预处理并存储。
    - 小数据集：当有新数据时重新处理并存储。
- 上游：数据在馈送模型之前在模型上游被预处理。
    - 在 CPU 上训练。
- 下游：数据在馈送模型之前在模型下游被预处理。
    - 在硬件加速器上训练（例如，GPU/TPU）。

#### 模型馈送

数据用于模型馈送可以是：

- 内存中：小数据集。
- 从磁盘读取：大数据集，快速训练。
- 从磁盘读取的 `Dataflow`：大规模数据集，扩展训练。

#### AutoML

对于 AutoML 训练，预处理和模型馈送会自动处理。

另外对于 AutoML 表格模型训练，您可以重新配置默认的预处理。

### 数据集

本教程使用的数据集是来自[BigQuery公共数据集](https://cloud.google.com/bigquery/public-data)的GSOD数据集。您使用的数据集版本只使用年份、月份和日期字段来预测每日平均温度（mean_temp）的值。

###  费用
本教程使用 Google Cloud 的计费组件：

- Vertex AI
- Cloud Storage
- BigQuery
- Dataflow

了解[Vertex AI 定价](https://cloud.google.com/vertex-ai/pricing)、[Cloud Storage 定价](https://cloud.google.com/storage/pricing)、[BigQuery 定价](https://cloud.google.com/bigquery/pricing) 和[Dataflow 定价](https://cloud.google.com/dataflow/pricing)，并使用[定价计算器](https://cloud.google.com/products/calculator/)根据您的预计使用量生成成本估算。

安装以下软件包以执行此笔记本。

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME") and not os.getenv("VIRTUAL_ENV")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

extra_pkgs = "tensorflow==2.5 tensorflow-data-validation==1.2 tensorflow-transform==1.2 \
              tensorflow-io==0.18 pyarrow pandas apache-beam[gcp] google-cloud-bigquery"
! pip3 install --upgrade --quiet {USER_FLAG} google-cloud-aiplatform $extra_pkgs

重新启动内核

安装完额外的包后，您需要重新启动笔记本内核，以便它可以找到这些包。

In [ ]:
import sys

if "google.colab" in sys.modules:
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

常见设置

现在，执行笔记本教程的常见设置。

In [ ]:
# Common code setup for notebook tutorials

! wget https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/ml_ops/setup.py -O setup.py

%run setup.py --bucket

In [ ]:
# Other Common setup instructions for notebook tutorials

! wget https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/ml_ops/setup.md -O setup.md

%load setup.md 

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常量

In [ ]:
import google.cloud.aiplatform as aiplatform

#### 导入Apache Beam

将Apache Beam包导入到您的Python环境中。

In [ ]:
import apache_beam as beam

#### 导入 BigQuery

将 BigQuery 包导入到您的 Python 环境中。

In [ ]:
from google.cloud import bigquery

#### 导入pandas

将pandas包导入到您的Python环境中。

In [ ]:
import pandas as pd

导入numpy

将numpy包导入到您的Python环境中。

In [ ]:
import numpy as np

导入 TensorFlow 数据验证

将 TensorFlow 数据验证（TFDV）包导入到您的Python环境中。

In [ ]:
import tensorflow_data_validation as tfdv

#### 导入TensorFlow变换

在您的Python环境中导入TensorFlow变换（TFT）包。

In [ ]:
import tensorflow_transform as tft

### 初始化Python的Vertex AI SDK

为您的项目和相应的存储桶初始化Python的Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION)

创建BigQuery客户端。

In [ ]:
bqclient = bigquery.Client()

## 使用 pandas dataframe 对 BigQuery 表进行离线预处理数据

- 离线：在训练之前，BigQuery 表在内存中进行预处理并存储。

    - 将表格数据提取到 pandas dataframe 中。
    - 在 dataframe 中逐列对数据进行预处理。
    - 将预处理后的 dataframe 写入新的 BigQuery 表中。

In [ ]:
IMPORT_FILE = "bq://bigquery-public-data.samples.gsod"
BQ_TABLE = "bigquery-public-data.samples.gsod"

### 将BigQuery数据集读入pandas dataframe

接下来，您可以使用BigQuery的`list_rows()`和`to_dataframe()`方法，将数据集的样本读入pandas dataframe，具体如下：

- `list_rows()`: 在指定表上执行查询并返回一个行迭代器以获取查询结果。可选指定：
   - `selected_fields`: 要返回的字段（列）的子集。
   - `max_results`: 要返回的行数的最大值。与SQL LIMIT命令相同。

- `rows.to_dataframe()`: 调用行迭代器并将数据读入pandas dataframe。

了解更多关于[将BigQuery表加载到dataframe](https://cloud.google.com/bigquery/docs/bigquery-storage-python-pandas)的信息。

In [ ]:
# Download a table.
table = bigquery.TableReference.from_string("bigquery-public-data.samples.gsod")

rows = bqclient.list_rows(
    table,
    max_results=500,
    selected_fields=[
        bigquery.SchemaField("station_number", "STRING"),
        bigquery.SchemaField("year", "INTEGER"),
        bigquery.SchemaField("month", "INTEGER"),
        bigquery.SchemaField("day", "INTEGER"),
        bigquery.SchemaField("mean_temp", "FLOAT"),
    ],
)

dataframe = rows.to_dataframe()
print(dataframe.head())

### 在pandas dataframe中转换数据。

接下来，你需要对dataframe中的数据进行预处理。

In [ ]:
dataframe["station_number"] = pd.to_numeric(dataframe["station_number"])

### 创建BQ数据集资源

首先，在您的项目中创建一个空的数据集资源。

In [ ]:
BQ_MY_DATASET = 'samples'
BQ_MY_TABLE = 'gsod'
! bq --location=US mk -d \
$PROJECT_ID:$BQ_MY_DATASET

In [ ]:
job_config = bigquery.LoadJobConfig(
    # Specify a (partial) schema. All columns are always written to the
    # table. The schema is used to assist in data type definitions.
    schema=[
        bigquery.SchemaField("station_number", "FLOAT"),  # <-- after one hot encoding
        bigquery.SchemaField("year", "INTEGER"),
        bigquery.SchemaField("month", "INTEGER"),
        bigquery.SchemaField("day", "INTEGER"),
        bigquery.SchemaField("mean_temp", "FLOAT"),
    ],
    # Optionally, set the write disposition. BigQuery appends loaded rows
    # to an existing table by default, but with WRITE_TRUNCATE write
    # disposition it replaces the table with the loaded data.
    write_disposition="WRITE_TRUNCATE",
)

NEW_BQ_TABLE = f"{PROJECT_ID}.samples.gsod_transformed"

job = bqclient.load_table_from_dataframe(
    dataframe, NEW_BQ_TABLE, job_config=job_config
)  # Make an API request.
job.result()  # Wait for the job to complete.

table = bqclient.get_table(NEW_BQ_TABLE)  # Make an API request.
print(
    "Loaded {} rows and {} columns to {}".format(
        table.num_rows, len(table.schema), NEW_BQ_TABLE
    )
)

## 使用 tf.data.Dataset 生成器对上游数据进行预处理

### 图像数据

- 上游数据：数据在模型训练时被提前进行预处理。

    - 定义预处理函数：
        - 输入：未处理的张量批量
        - 输出：预处理后的张量批量
    - 使用 tf.data.Dataset 的 `map()` 方法将预处理函数映射到生成器的输出中。

在本例中：

- 将 CIFAR10 数据集加载到内存中作为 numpy 数组。
- 为内存中的 CIFAR10 数据集创建一个 tf.data.Dataset 生成器。*注意*：将像素数据转换为 FLOAT32 类型，以便与将像素数据输出为 FLOAT32 类型的预处理函数兼容。
- 定义一个预处理函数来通过 1/255.0 对像素数据进行重新缩放。
- 将预处理函数映射到生成器中。

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

tf_dataset = tf.data.Dataset.from_tensor_slices((x_train.astype(np.float32), y_train))

print("Before preprocessing")
for batch in tf_dataset:
    print(batch)
    break


def preprocess_fn(inputs, labels):
    inputs /= 255.0
    return tf.cast(inputs, tf.float32), labels


tf_dataset = tf_dataset.map(preprocess_fn)

print("After preprocessing")
for batch in tf_dataset:
    print(batch)
    break

## 通过tf.data.Dataset生成器进行上游预处理数据

### 表格数据

- 上游: 在数据被用于训练之前，数据被模型上游预处理。

    - 定义预处理函数:
        - 输入: 未加工的张量批次
        - 输出: 预处理的张量批次
    - 使用tf.data.Dataset `map()` 方法将预处理函数映射到生成器输出。

在这个例子中:

- 为波士顿房屋数据创建tf.data.Dataset生成器。
- 在预处理前迭代一个批次。
- 定义预处理函数，将所有特征缩放在0到1之间。
- 将预处理函数映射到数据集。
- 通过数据集迭代一次。

In [ ]:
from tensorflow.keras.datasets import boston_housing

(x_train, y_train), (x_test, y_test) = boston_housing.load_data()

tf_dataset = tf.data.Dataset.from_tensor_slices((x_train.astype(np.float32), y_train))

print("Before preprocessing")
for batch in tf_dataset:
    print(batch)
    break


def preprocessing_fn(inputs, labels):
    inputs = tft.scale_to_0_1(inputs)
    return tf.cast(inputs, tf.float32), labels


tf_dataset = tf_dataset.map(preprocessing_fn)

print("After preprocessing")
for batch in tf_dataset:
    print(batch)
    break

## 数据流离线预处理

- 从BigQuery表生成数据架构。
- 定义Beam管道来：
    - 将BigQuery表中的数据分割成训练集和评估集。
    - 使用数据架构将数据集编码为TFRecords。
    - 将TFRecords保存为压缩文件到Cloud Storage。
- 运行管道。

### 将BigQuery数据集读取到pandas dataframe中

接下来，您可以使用BigQuery `list_rows()` 和 `to_dataframe()` 方法从数据集中读取一部分数据到pandas dataframe中，具体步骤如下：

- `list_rows()`: 在指定表上执行查询并返回一个行迭代器来获取查询结果。您可以选择指定以下参数：
 - `selected_fields`: 要返回的字段（列）的子集。
 - `max_results`: 要返回的最大行数。类似于SQL的LIMIT命令。

- `rows.to_dataframe()`: 调用行迭代器并将数据读取到pandas dataframe中。

了解更多关于[将BigQuery表加载到dataframe中](https://cloud.google.com/bigquery/docs/bigquery-storage-python-pandas)。

In [ ]:
# Download a table.
table = bigquery.TableReference.from_string("bigquery-public-data.samples.gsod")

rows = bqclient.list_rows(
    table,
    max_results=500,
    selected_fields=[
        bigquery.SchemaField("station_number", "STRING"),
        bigquery.SchemaField("year", "INTEGER"),
        bigquery.SchemaField("month", "INTEGER"),
        bigquery.SchemaField("day", "INTEGER"),
        bigquery.SchemaField("mean_temp", "FLOAT"),
    ],
)

dataframe = rows.to_dataframe()
print(dataframe.head())

### 生成数据集统计信息

#### Dataframe 输入数据

使用 TensorFlow Data Validation (TFDV) 包对数据集生成统计信息。使用 `generate_statistics_from_dataframe()` 方法，其中包括以下参数：

- `dataframe`: 存储在内存中的 pandas dataframe 格式的数据集。
- `stats_options`: 选择的统计选项:
  - `label_feature`: 要预测的列。
  - `sample_rate`: 采样率。如果指定了该参数，将在采样数据上计算统计信息。
  - `num_top_values`: 对于字符串类型特征，保留的最常见特征值数量。

了解有关[TensorFlow Data Validation (TFDV)](https://www.tensorflow.org/tfx/data_validation/get_started)的信息。

In [ ]:
stats = tfdv.generate_statistics_from_dataframe(
    dataframe=dataframe,
    stats_options=tfdv.StatsOptions(
        label_feature="mean_temp", sample_rate=1, num_top_values=50
    ),
)

print(stats)

生成原始数据架构

使用 TensorFlow Data Validation (TFDV) 包在数据集上生成数据架构。使用 `infer_schema()` 方法，使用以下参数：

- `statistics`: TFDV 生成的统计数据。

In [ ]:
schema = tfdv.infer_schema(statistics=stats)
print(schema)

#### 将数据集的架构保存到云存储

接下来，将数据集的架构写入数据集的云存储存储桶中。

In [ ]:
SCHEMA_LOCATION = BUCKET_URI + "/schema.txt"

# When running Apache Beam directly (file is directly accessed)
tfdv.write_schema_text(output_path=SCHEMA_LOCATION, schema=schema)
# When running with Dataflow (file is uploaded to worker pool)
tfdv.write_schema_text(output_path="schema.txt", schema=schema)

#### 为Dataflow作业准备软件包要求。

在您运行Dataflow作业之前，您需要为将执行作业的工作池指定软件包要求。

In [ ]:
%%writefile setup.py
import setuptools

REQUIRED_PACKAGES = [
    "google-cloud-aiplatform==1.4.2",
    "tensorflow-transform==1.2.0",
    "tensorflow-data-validation==1.2.0",
]

setuptools.setup(
    name="executor",
    version="0.0.1",
    install_requires=REQUIRED_PACKAGES,
    packages=setuptools.find_packages(),
    include_package_data=True,
    package_data={"./": ["schema.txt"]}
)

### 使用Dataflow对数据进行预处理

#### 数据集拆分

接下来，您将使用Dataflow对数据进行预处理。在本例中，您将查询BigQuery表并将示例拆分为训练和评估数据集。出于便利起见，数据集中的示例数量被限制为500个。

In [ ]:
import os

import tensorflow_transform.beam as tft_beam

RUNNER = "DataflowRunner"  # DirectRunner for local running w/o Dataflow


def parse_bq_record(bq_record):
    """Parses a bq_record to a dictionary."""
    output = {}
    for key in bq_record:
        output[key] = [bq_record[key]]
    return output


def split_dataset(bq_row, num_partitions, ratio):
    """Returns a partition number for a given bq_row."""
    import json

    assert num_partitions == len(ratio)
    bucket = sum(map(ord, json.dumps(bq_row))) % sum(ratio)
    total = 0
    for i, part in enumerate(ratio):
        total += part
        if bucket < total:
            return i
    return len(ratio) - 1


def run_pipeline(args):
    """Runs a Beam pipeline to split the dataset"""

    pipeline_options = beam.pipeline.PipelineOptions(flags=[], **args)

    raw_data_query = args["raw_data_query"]
    exported_data_prefix = args["exported_data_prefix"]
    temp_location = args["temp_location"]
    project = args["project"]

    schema = tfdv.load_schema_text(SCHEMA_LOCATION)

    with beam.Pipeline(options=pipeline_options) as pipeline:
        with tft_beam.Context(temp_location):

            # Read raw BigQuery data.
            raw_train_data, raw_eval_data = (
                pipeline
                | "Read Raw Data"
                >> beam.io.ReadFromBigQuery(
                    query=raw_data_query,
                    project=project,
                    use_standard_sql=True,
                )
                | "Parse Data" >> beam.Map(parse_bq_record)
                | "Split" >> beam.Partition(split_dataset, 2, ratio=[8, 2])
            )

            _ = (
                raw_train_data
                | "Write Raw Train Data"
                >> beam.io.tfrecordio.WriteToTFRecord(
                    file_path_prefix=os.path.join(exported_data_prefix, "train/"),
                    file_name_suffix=".gz",
                    coder=tft.coders.ExampleProtoCoder(schema),
                )
            )

            _ = (
                raw_eval_data
                | "Write Raw Eval Data"
                >> beam.io.tfrecordio.WriteToTFRecord(
                    file_path_prefix=os.path.join(exported_data_prefix, "eval/"),
                    file_name_suffix=".gz",
                    coder=tft.coders.ExampleProtoCoder(schema),
                )
            )


EXPORTED_DATA_PREFIX = os.path.join(BUCKET_URI, "exported_data")

QUERY_STRING = "SELECT {},{} FROM {} LIMIT 500".format(
    "CAST(station_number as STRING) AS station_number,year,month,day",
    "mean_temp",
    IMPORT_FILE[5:],
)
JOB_NAME = "gsod" + TIMESTAMP

args = {
    "runner": RUNNER,
    "raw_data_query": QUERY_STRING,
    "exported_data_prefix": EXPORTED_DATA_PREFIX,
    "temp_location": os.path.join(BUCKET_URI, "temp"),
    "project": PROJECT_ID,
    "region": REGION,
    "setup_file": "./setup.py",
}

print("Data preprocessing started...")
run_pipeline(args)
print("Data preprocessing completed.")

! gsutil ls $EXPORTED_DATA_PREFIX/train
! gsutil ls $EXPORTED_DATA_PREFIX/eval

清理

要清理此项目中使用的所有谷歌云资源，您可以[删除用于本教程的谷歌云项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除您在本教程中创建的各个资源。

In [ ]:
delete_storage = False

if delete_storage or os.getenv("IS_TESTING"):
    if "BUCKET_URI" in globals():
        ! gsutil rm -r $BUCKET_URI